In [1]:
import pandas as pd
import pypyodbc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
server = 'LIELAH\SQLEXPRESS' 
database = 'foodDelivery'   
cnxn = pypyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database)
cursor = cnxn.cursor()
query = "SELECT * from foodDelivery;"
df = pd.read_sql(query, cnxn)
df.head()

c:\Users\smabh\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,deliverypersonid,deliverypersonage,deliverypersonratings,orderdate,timeorderd,weatherconditions,roadtrafficdensity,vehiclecondition,typeoforder,typeofvehicle,multipledeliveries,festival,city,timetaken,pickuptime,delivereddistance
0,0x4607,INDORES13DEL02,37.0,4.9,2022-03-19,11:30:00.0000000,conditions Sunny,High,2,Snack,motorcycle,0.0,False,Urban,24,15.0,0.028282
1,0xb379,BANGRES18DEL02,34.0,4.5,2022-03-25,19:45:00.0000000,conditions Stormy,Jam,2,Snack,scooter,1.0,False,Metropolitian,33,5.0,0.183851
2,0x5d6d,BANGRES19DEL01,23.0,4.4,2022-03-19,08:30:00.0000000,conditions Sandstorms,Low,0,Drinks,motorcycle,1.0,False,Urban,26,15.0,0.014144
3,0x7a6a,COIMBRES13DEL02,38.0,4.7,2022-04-05,18:00:00.0000000,conditions Sunny,Medium,0,Buffet,motorcycle,1.0,False,Metropolitian,21,10.0,0.070713
4,0x70a2,CHENRES12DEL01,32.0,4.6,2022-03-26,13:30:00.0000000,conditions Cloudy,High,1,Snack,scooter,1.0,False,Metropolitian,30,15.0,0.056569


In [2]:
print(df.dtypes)
print('         ---null data---')
print(df.isna().sum())
df.head()
df['timetaken'] = df['timetaken'].astype(float)
df['delivereddistance'] = df['delivereddistance'].astype(float)
df['multipledeliveries'] = df['multipledeliveries'].astype(float)
df.deliverypersonage = df.deliverypersonage.fillna(df.deliverypersonage.mean())
df.deliverypersonratings = df.deliverypersonratings.fillna(df.deliverypersonratings.mean())
df.pickuptime = df.pickuptime.fillna(df.pickuptime.mode()[0])
df.city = df.city.fillna(df.city.mode()[0])
df.festival = df.festival.fillna(df.festival.mode()[0])
df.multipledeliveries = df.multipledeliveries.fillna(df.multipledeliveries.mode()[0])
df.roadtrafficdensity = df.roadtrafficdensity.fillna(df.roadtrafficdensity.mode()[0])
df['orderdate'] =  pd.to_datetime(df['orderdate'], format='%Y/%m/%d')
df['orderedmonth'] = df['orderdate'].dt.strftime('%m-%y')
df = df.dropna()
df['timeorderd'] = df['timeorderd'].astype(str)
df['timeorderd'] = df['timeorderd'].str.split('.').str[0]
df['timeorderd'] = pd.to_datetime(df['timeorderd'],format= '%H:%M:%S' ).dt.time
df.head()

id                        object
deliverypersonid          object
deliverypersonage        float64
deliverypersonratings    float64
orderdate                 object
timeorderd                object
weatherconditions         object
roadtrafficdensity        object
vehiclecondition           int64
typeoforder               object
typeofvehicle             object
multipledeliveries       float64
festival                  object
city                      object
timetaken                 object
pickuptime               float64
delivereddistance        float64
dtype: object
         ---null data---
id                          0
deliverypersonid            0
deliverypersonage        1611
deliverypersonratings    1660
orderdate                   0
timeorderd               1456
weatherconditions           0
roadtrafficdensity        454
vehiclecondition            0
typeoforder                 0
typeofvehicle               0
multipledeliveries        985
festival                  228
city      

,id,deliverypersonid,deliverypersonage,deliverypersonratings,orderdate,timeorderd,weatherconditions,roadtrafficdensity,vehiclecondition,typeoforder,typeofvehicle,multipledeliveries,festival,city,timetaken,pickuptime,delivereddistance,orderedmonth
0,0x4607,INDORES13DEL02,37.0,4.9,2022-03-19,11:30:00,conditions Sunny,High,2,Snack,motorcycle,0.0,False,Urban,24.0,15.0,0.028282,03-22
1,0xb379,BANGRES18DEL02,34.0,4.5,2022-03-25,19:45:00,conditions Stormy,Jam,2,Snack,scooter,1.0,False,Metropolitian,33.0,5.0,0.183851,03-22
2,0x5d6d,BANGRES19DEL01,23.0,4.4,2022-03-19,08:30:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1.0,False,Urban,26.0,15.0,0.014144,03-22
3,0x7a6a,COIMBRES13DEL02,38.0,4.7,2022-04-05,18:00:00,conditions Sunny,Medium,0,Buffet,motorcycle,1.0,False,Metropolitian,21.0,10.0,0.070713,04-22
4,0x70a2,CHENRES12DEL01,32.0,4.6,2022-03-26,13:30:00,conditions Cloudy,High,1,Snack,scooter,1.0,False,Metropolitian,30.0,15.0,0.056569,03-22


In [3]:
age_bucket = [18, 25, 30, 35, 40, 45]
df['ages'] = pd.cut(df['deliverypersonage'], age_bucket)
delivery_count = df.groupby(['ages']).count()
delivery_count = delivery_count['deliverypersonage']
delivery_count = delivery_count.to_frame('delivery_count')
delivery_count = delivery_count.reset_index()
delivery_count['ages'] = delivery_count['ages'].astype(str)
delivery_count

,ages,delivery_count
0,"(18, 25]",12916
1,"(25, 30]",11070
2,"(30, 35]",10900
3,"(35, 40]",8820
4,"(40, 45]",0


In [4]:
total_TimeAndDistance = df.groupby(['city','typeofvehicle']).sum()
total_TimeAndDistance = total_TimeAndDistance.drop(['deliverypersonage','deliverypersonratings','vehiclecondition','multipledeliveries', 'festival','pickuptime'], axis =1)
total_TimeAndDistance = total_TimeAndDistance.reset_index()
total_TimeAndDistance

,city,typeofvehicle,timetaken,delivereddistance
0,Metropolitian,electric_scooter,68463.0,248.463187
1,Metropolitian,motorcycle,566359.0,1817.715520
2,Metropolitian,scooter,283305.0,1030.126651
3,Semi-Urban,electric_scooter,147.0,0.325266
4,Semi-Urban,motorcycle,7138.0,16.857426
5,Semi-Urban,scooter,474.0,1.541494
6,Urban,electric_scooter,17365.0,67.698326
7,Urban,motorcycle,131423.0,456.352260
8,Urban,scooter,74713.0,292.063220


In [5]:
totalVehicleCount = df.groupby(['city','typeofvehicle']).count()
totalVehicleCount = totalVehicleCount.drop(['deliverypersonid', 'deliverypersonage',
       'deliverypersonratings', 'orderdate', 'timeorderd',
       'weatherconditions', 'roadtrafficdensity', 'vehiclecondition',
       'typeoforder', 'multipledeliveries', 'festival', 'timetaken',
       'pickuptime', 'delivereddistance', 'ages','orderedmonth'], axis =1)
totalVehicleCount = totalVehicleCount.reset_index()
totalVehicleCount

,city,typeofvehicle,id
0,Metropolitian,electric_scooter,2709
1,Metropolitian,motorcycle,19989
2,Metropolitian,scooter,11133
3,Semi-Urban,electric_scooter,3
4,Semi-Urban,motorcycle,143
5,Semi-Urban,scooter,10
6,Urban,electric_scooter,808
7,Urban,motorcycle,5405
8,Urban,scooter,3506


In [7]:
for index, row in delivery_count.iterrows():
    cursor.execute("INSERT INTO AgedDeliveryCount (ages, deliveriesCount) Values(?,?);", [row.ages, row.delivery_count])
cnxn.commit()
for index, row in total_TimeAndDistance.iterrows():
    cursor.execute("INSERT INTO  totalTimeAndDistance(city, typeOfVehicles, timeTaken, deliveredDistance) Values(?,?,?,?);", 
    [row.city,row.typeofvehicle,row.timetaken,row.delivereddistance])
cnxn.commit()
for index, row in totalVehicleCount.iterrows():
    cursor.execute("INSERT INTO totalVehiclesCount (city,typeOfVehicles, cnt) Values(?,?,?);", [row.city, row.typeofvehicle,row.id])
cnxn.commit()
cursor.close()


In [8]:
df.columns

Index(['id', 'deliverypersonid', 'deliverypersonage', 'deliverypersonratings',
       'orderdate', 'timeorderd', 'weatherconditions', 'roadtrafficdensity',
       'vehiclecondition', 'typeoforder', 'typeofvehicle',
       'multipledeliveries', 'festival', 'city', 'timetaken', 'pickuptime',
       'delivereddistance', 'orderedmonth', 'ages'],
      dtype='object')

In [6]:
df = df.drop(['id', 'deliverypersonid', 'deliverypersonage', 'deliverypersonratings',
       'orderdate', 'timeorderd', 'weatherconditions', 'roadtrafficdensity',
       'vehiclecondition', 'typeoforder', 'typeofvehicle',
       'multipledeliveries', 'festival', 'city', 'timetaken', 'pickuptime',
       'delivereddistance', 'orderedmonth', 'ages'])

,id,delivery_person_id,delivery_person_age,delivery_person_ratings,order_date,time_orderd,weatherconditions,road_traffic_density,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city,time_taken_min,pickuptime,delivereddistance,orderedmonth,ages
0,0x5b31,VADRES11DEL01,29.0,4.5,2022-04-03,11:15:00,conditions Sandstorms,High,1,Drinks,scooter,0.0,False,Metropolitian,24.0,10.0,0.014138,04-22,"(25, 30]"
1,0x5e9f,CHENRES08DEL03,37.0,4.9,2022-04-05,19:45:00,conditions Sunny,Jam,2,Drinks,scooter,1.0,False,Urban,21.0,15.0,0.084851,04-22,"(35, 40]"
2,0xc706,KOLRES16DEL03,21.0,4.5,2022-02-15,19:15:00,conditions Stormy,Jam,1,Meal,scooter,1.0,False,Metropolitian,23.0,10.0,0.084851,02-22,"(18, 25]"
3,0x3caa,INDORES18DEL01,25.0,4.8,2022-03-14,23:45:00,conditions Fog,Low,2,Snack,electric_scooter,0.0,False,Metropolitian,20.0,15.0,0.098995,03-22,"(18, 25]"
4,0xb7d7,SURRES03DEL01,20.0,4.8,2022-03-31,23:45:00,conditions Cloudy,Low,1,Buffet,motorcycle,0.0,False,Metropolitian,22.0,15.0,0.098995,03-22,"(18, 25]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45157,0x6acc,INDORES05DEL02,20.0,4.6,2022-03-19,20:25:00,conditions Cloudy,Jam,2,Meal,scooter,0.0,False,Metropolitian,20.0,10.0,0.070708,03-22,"(18, 25]"
45158,0xb5ce,INDORES20DEL02,32.0,4.7,2022-04-05,19:40:00,conditions Sunny,Jam,2,Buffet,scooter,0.0,False,Metropolitian,22.0,15.0,0.070712,04-22,"(30, 35]"
45159,0x7d1d,HYDRES03DEL03,29.0,4.0,2022-03-03,20:15:00,conditions Cloudy,Jam,1,Drinks,motorcycle,1.0,False,Metropolitian,33.0,5.0,0.084851,03-22,"(25, 30]"
45160,0x5c6c,PUNERES05DEL03,35.0,4.6,2022-03-16,17:35:00,conditions Sunny,Medium,0,Buffet,motorcycle,1.0,False,Metropolitian,29.0,15.0,0.127282,03-22,"(30, 35]"


In [ ]:
city_df = pd.get_dummies(df['City'], drop_first= True)
festival_df = pd.get_dummies(df['Festival'], drop_first= True)
vehicle_df = pd.get_dummies(df['Type_of_vehicle'], drop_first= True)
order_df = pd.get_dummies(df['Type_of_order'], drop_first= True)
traffic_density_df = pd.get_dummies(df['Road_traffic_density'], drop_first= True)
weather_df = pd.get_dummies(df['Weatherconditions'], drop_first= True)
df = pd.concat([df,city_df,festival_df,vehicle_df,order_df,traffic_density_df,weather_df], axis = 1)
df